In [1]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import loompy as lp

In [2]:
#adata.write('/home/julia/data/placodes/h5ad/11_normalized.h5ad')

In [2]:
adata=sc.read_h5ad('/home/julia/data/placodes/h5ad/11_qc.h5ad')

/home/julia/miniconda3/envs/scenicplus/lib/python3.8/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [3]:
#adata=sc.read_h5ad('/home/julia/data/placodes/h5ad/11_normalized.h5ad')

In [4]:
adata.obs_names_make_unique()

In [5]:
#adata=sc.read_h5ad('/home/julia/data/placodes/noutbooks/h5ad_files/14_06_ALL_7_datasets_qc.h5ad')

In [6]:
adata

AnnData object with n_obs × n_vars = 62269 × 25865
    obs: 'plate', 'day', 'location', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes', 'doublet_score', 'predicted_doublet', 'batch'
    layers: 'counts'

In [7]:
adata_placodes=sc.read_h5ad('/home/julia/data/placodes/ANDYs_notebboks/h5ad/14_06_all_placods_epcam_subset.h5ad')

/home/julia/miniconda3/envs/scenicplus/lib/python3.8/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [8]:
adata_placodes

AnnData object with n_obs × n_vars = 42129 × 25865
    obs: 'plate', 'day', 'location', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes', 'doublet_score', 'predicted_doublet', 'batch', 'leiden'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'day_colors', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    obsp: 'connectivities', 'distances'

In [9]:
adata_placodes.obs_names_make_unique()

In [10]:
cell=adata_placodes.obs_names
adata[cell]

View of AnnData object with n_obs × n_vars = 42129 × 25865
    obs: 'plate', 'day', 'location', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes', 'doublet_score', 'predicted_doublet', 'batch'
    layers: 'counts'

In [11]:
adata_new=adata[cell]

In [12]:
adata_new

View of AnnData object with n_obs × n_vars = 42129 × 25865
    obs: 'plate', 'day', 'location', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes', 'doublet_score', 'predicted_doublet', 'batch'
    layers: 'counts'

In [13]:
adata_new.write('adata_placodes.h5ad')

In [14]:
adata=adata_new

In [15]:
adata

View of AnnData object with n_obs × n_vars = 42129 × 25865
    obs: 'plate', 'day', 'location', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes', 'doublet_score', 'predicted_doublet', 'batch'
    layers: 'counts'

In [22]:

# path to unfiltered loom file (this will be created in the optional steps below)
f_loom_path_unfilt = "placods_unfiltered.loom" # test dataset, n=500 cells

# # path to loom file with basic filtering applied (this will be created in the "initial filtering" step below). Optional.
f_loom_path_scenic = "placodes_filtered_scenic.loom"

# path to anndata object, which will be updated to store Scanpy results as they are generated below
#f_anndata_path = "anndata.h5ad"

# path to pyscenic output
f_pyscenic_output = "placodes_pyscenic_output.loom"

# loom output, generated from a combination of Scanpy and pySCENIC results:
f_final_loom = 'placodes_scenic_integrated-output.loom'

In [23]:
adata.obs_names

Index(['AAACGAAGTTGAGAGC-1', 'AAACGCTGTCCGACGT-1', 'AAAGGATTCAAGTTGC-1',
       'AAAGGTAAGGGTACGT-1', 'AAAGGTACAGTCAGTT-1', 'AAAGTGACACGATTCA-1',
       'AACAAAGCATCCGGTG-1', 'AACAAAGTCATTCCTA-1', 'AACAAGAAGCGACCCT-1',
       'AACAAGATCGAGCCTG-1',
       ...
       'TTTGATCTCTGAACGT-1-3', 'TTTGGAGGTTAATCGC-1-3', 'TTTGGTTCATGACAAA-1-3',
       'TTTGGTTGTAGTCTTG-1-3', 'TTTGGTTGTCTACAGT-1-3', 'TTTGGTTTCCTTACCG-1-3',
       'TTTGTTGAGCAAGCCA-1-3', 'TTTGTTGAGGCAGCTA-1-3', 'TTTGTTGCATTCTTCA-1-3',
       'TTTGTTGTCCTAAACG-1-3'],
      dtype='object', length=42129)

In [24]:
# create basic row and column attributes for the loom file:
row_attrs = {
    "Gene": np.array(adata.var_names) ,
}
col_attrs = {
    "CellID": np.array(adata.obs_names) ,
    "nGene": np.array( np.sum(adata.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(adata.X.transpose() , axis=0)).flatten() ,
}
lp.create( f_loom_path_scenic, adata.X.transpose(), row_attrs, col_attrs)

In [21]:
f_tfs='/home/julia/data/Scenic_database/Mouse/TF/allTFs_mm.txt'

In [22]:
!pyscenic grn {f_loom_path_scenic} {f_tfs} -o adj.csv --num_workers 19



2024-01-12 10:05:53,207 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2024-01-12 10:06:06,638 - pyscenic.cli.pyscenic - INFO - Inferring regulatory networks.
preparing dask client
parsing input
creating dask graph
19 partitions
computing dask graph
not shutting down client, client was created externally
finished

2024-01-13 08:48:43,819 - pyscenic.cli.pyscenic - INFO - Writing results to file.


In [16]:
adjacencies = pd.read_csv("adj.csv", index_col=False)

In [17]:
adjacencies.head()

,TF,target,importance
0,Rps10,Eef1a1,404.295748
1,Rps4x,Eef1a1,378.464830
2,Hmgb2,Ube2c,369.717699
3,Hand1,Slc2a1,367.812258
4,Trp63,Pdgfa,356.626697


In [25]:
import glob
# ranking databases
f_db_glob = "/home/julia/data/Scenic_database/Mouse/Gene_based/*feather"
f_db_names = ' '.join( glob.glob(f_db_glob) )

In [26]:
# motif databases
f_motif_path = "/home/julia/data/Scenic_database/Mouse/Motif2TF/motifs-v10nr_clust-nr.mgi-m0.001-o0.0.tbl"

In [27]:
!pyscenic ctx adj.csv \
    {f_db_names} \
    --annotations_fname {f_motif_path} \
    --expression_mtx_fname {f_loom_path_scenic} \
    --output reg.csv \
    --mask_dropouts \
    --num_workers 16


2024-01-15 10:18:59,815 - pyscenic.cli.pyscenic - INFO - Creating modules.

2024-01-15 10:19:02,894 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2024-01-15 10:19:20,336 - pyscenic.utils - INFO - Calculating Pearson correlations.

2024-01-15 10:19:21,899 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [True].

2024-01-15 10:23:16,433 - pyscenic.utils - INFO - Creating modules.

2024-01-15 10:26:29,969 - pyscenic.cli.pyscenic - IN

In [28]:
!pyscenic aucell \
    {f_loom_path_scenic} \
    reg.csv \
    --output {f_pyscenic_output} \
    --num_workers 20


2024-01-15 11:40:40,410 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2024-01-15 11:40:54,003 - pyscenic.cli.pyscenic - INFO - Loading gene signatures.
Create regulons from a dataframe of enriched features.
Additional columns saved: []

2024-01-15 11:40:59,722 - pyscenic.cli.pyscenic - INFO - Calculating cellular enrichment.

2024-01-15 11:43:38,274 - pyscenic.cli.pyscenic - INFO - Writing results to file.
